In [0]:
# service principal credentials
client_id = dbutils.secrets.get("formula1-scope", "formula1-data-platform-client-id")
tenant_id = dbutils.secrets.get("formula1-scope", "formula1-data-platform-tenant-id")
client_secret = dbutils.secrets.get("formula1-scope", "formula1-data-platform-client-secret")

In [0]:
# storage account variables
storage_account_name = "formula1dataplatformdl"
containers = ["demo", "presentation", "processed", "raw"]

In [0]:
# mount config
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": client_id,
           "fs.azure.account.oauth2.client.secret": client_secret,
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

In [0]:
# function to mount adls
def mount_container(storage_account: str, container: str):
    for mount in dbutils.fs.mounts():
        if mount.mountPoint == f"/mnt/{storage_account}/{container}":
            print(f"Container {storage_account}/{container} already mounted")
            return

    dbutils.fs.mount(
        source = f"abfss://{container}@{storage_account}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account}/{container}",
        extra_configs = configs
    )
    print(f"Mounted {storage_account}/{container} as /mnt/{storage_account}/{container}")

In [0]:
for container in containers:
    mount_container(storage_account_name, container)
display(dbutils.fs.mounts())